In [1]:
knitr::opts_chunk$set(echo = TRUE)
require(here)
require(data.table)
require(tidyverse)
require(reshape2)
require(ggpubr)
require(cowplot)
require(coin)


Loading required package: here

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘here’”
Loading required package: data.table

Loading required package: tidyverse

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.3     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr::transpose() masks data.table::transpose()

Loading required package: reshape2


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


The following objects are masked 

In [ ]:
knitr::opts_chunk$set(echo = TRUE)
dd_genes <- fread(here('data/discordant_consensus_novel_DD_associated_genes_kaplanis2020.txt'),header=FALSE)[,V1]
hotspot_genes <- fread(here('data/hotspot_genes.txt'),header=FALSE)[,V1]
novel_hotspot_genes <- fread(here('data/proposed_novel_hotspot_genes.txt'),header=FALSE)[,V1]
ion_channel_genes <- fread(here('data/All_genes_with_PF00520_domain.txt'),header=FALSE)[,V1]
hotspot_genes_lenient <- fread(here('data/lenient_hotspot_genes.txt'),header=FALSE)[,V1]
gene_tpm <- fread(here('data/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_median_tpm.gct.gz'),header=TRUE)
constraint_in <- fread(here('data/gnomad.v2.1.1.lof_metrics.by_gene.txt'),header=TRUE)
mpc_in <- fread(here('data/mpc_missense_constraint_multiple_regions_samocha17.txt'),header=TRUE)
regional_constraint <- fread(here('data/hotspot_mutation_regional_constraint.txt'))


**Input data and counts:**



In [ ]:
#Remove novel hotspot genes from hotspot gene list
hotspot_genes <- hotspot_genes[!hotspot_genes %in% novel_hotspot_genes]

#Remove hotspot genes from DD consensus gene list
dd_genes <- dd_genes[!dd_genes %in% hotspot_genes]

#Parse gene TPM file
gene_tpm <- gene_tpm[,Gene_ID:=gsub("\\..*","",Name)]
gene_tpm <- melt(gene_tpm,id.vars = c('Description','Name','Gene_ID'))
colnames(gene_tpm) <- c('hgnc','ensembl_versioned','ensembl','tissue','tpm')

gene_tpm <- gene_tpm[,class:=ifelse(hgnc %in% novel_hotspot_genes,'Proposed Novel Hotspot Genes',
                                    ifelse(hgnc %in% hotspot_genes,'Hotspot Genes',
                                           ifelse(hgnc %in% dd_genes,'DD Consensus Genes','Control Genes')))]

gene_tpm <- gene_tpm[,contains_ion_channel:=ifelse(hgnc %in% ion_channel_genes,TRUE,FALSE)]

head(gene_tpm)

gene_tpm_tab <- gene_tpm %>%
  group_by(class) %>%
  dplyr::select('hgnc','class') %>%
  unique() %>%
  count()

colnames(gene_tpm_tab) <- c('Class','Count')
  
#Table of counts by class (Control Genes, DD Consensus Genes, Hotspot Genes, Proposed Novel Hotspot Genes)
knitr::kable(gene_tpm_tab)


**Counts post-sampling:**



In [ ]:
#Set seed
set.seed(1255)

sampled_genes <- sample(unique(gene_tpm[class=='Control Genes',hgnc]),size=1000,replace=FALSE)

gene_tpm_sampled <- gene_tpm[class!='Control Genes' | hgnc %in% sampled_genes]

gene_tpm_sampled_tab <- gene_tpm_sampled %>%
  group_by(class) %>%
  dplyr::select('hgnc','class') %>%
  unique() %>%
  count()

colnames(gene_tpm_sampled_tab) <- c('Class','Count')

#Table of sampled counts by class (Control Genes, DD Consensus Genes, Hotspot Genes, Proposed Novel Hotspot Genes)
knitr::kable(gene_tpm_sampled_tab)


### Figure 4A: Hotspot genes share a characteristic gene expression pattern across tissues



In [ ]:
#Set seed
set.seed(1255)

#Get significant differences between groups by tissue; Bonferroni correct for multiple testing
getProportionP <- function( class1, class2 ) {
  
  temp <- gene_tpm_sampled[class==class1 | class==class2]
  temp <- temp[,expressed:=ifelse(tpm > 1, TRUE, FALSE)]
  temp <- temp[,c('hgnc','class','tissue','expressed')]
  
  fisherTest <- function( t ) {
    
    class1_temp <- temp[class==class1 & tissue==t]
    expressed_class1 <- nrow(class1_temp[which(expressed==TRUE)])
    unexpressed_class1 <- nrow(class1_temp[which(expressed==FALSE)])

    class2_temp <- temp[class==class2 & tissue==t]
    expressed_class2 <- nrow(class2_temp[which(expressed==TRUE)])
    unexpressed_class2 <- nrow(class2_temp[which(expressed==FALSE)])
  
    fisher_p <- fisher.test(cbind(c(expressed_class1,unexpressed_class1),
                    c(expressed_class2,unexpressed_class2)))$p.value
  
    return(fisher_p)
    
  }
  
  temp <- temp[,fisher_p:=fisherTest(tissue),by=tissue]
  temp <- unique(temp[,c('tissue','fisher_p')])
  temp <- temp[,fisher_corrected_p:=p.adjust(fisher_p,method='bonferroni')]
  return(temp)
  
}

getPermutationP <- function( class1, class2 ) {
  
  temp <- gene_tpm_sampled[ (class==class1 | class==class2)]
  temp <- temp[,c('class','tissue','tpm')]
  temp <- temp[,permutation_p:=pvalue(independence_test(tpm ~ factor(class))),by=tissue]
  temp <- unique(temp[,c('tissue','permutation_p')])
  temp <- temp[,permutation_corrected_p:=p.adjust(permutation_p,method='bonferroni')]
  return(temp)
  
}

dd_consensus_control_prop_table <- getProportionP('Hotspot Genes','DD Consensus Genes')
dd_consensus_control_tpm_table <- getPermutationP('DD Consensus Genes','Control Genes')
knitr::kable(dd_consensus_control_prop_table)
knitr::kable(dd_consensus_control_tpm_table)

#Randomly sample DD Consensus and Control Genes
sampleGenes <- function( iterations, sampling_size ) {

  out <- vector('list',length=iterations)
  
  for( i in 1:iterations ) {
    
    sampled_dat <- gene_tpm_sampled %>% 
    filter(class!='Hotspot Genes' & class!='Proposed Novel Hotspot Genes') %>%
    group_by(tissue,class) %>% 
    sample_n(sampling_size,replace=TRUE) %>%
    mutate(i=factor(i))
    
    out[[i]] <- data.table(sampled_dat)
    
  }
  
  out_dt <- do.call(rbind,out)
  return(out_dt)

}

hotspot_tpm <- gene_tpm_sampled %>%
  filter(class=='Hotspot Genes') %>%
  mutate(i=0)

#Bind sampled genes to hotspot genes; calculate the proportion expressed, mean, and standard deviation across sets
fig_a <- rbind(sampleGenes(1000,25),hotspot_tpm) %>%
  group_by(tissue,class,i) %>%
  summarise(
    prop_expressed = sum(tpm > 1, na.rm=T) / n(),
    .groups = 'drop_last'
  ) %>%
  ungroup() %>%
  group_by(tissue,class) %>%
  summarise(
    mean = mean(prop_expressed),
    min = mean - sd(prop_expressed),
    max = mean + sd(prop_expressed),
    .groups = 'keep'
  ) %>%
  data.table()

#Figure 4A
a_fill <- c('Control Genes' = 'skyblue3',
            'DD Consensus Genes' = 'darkgreen',
            'Hotspot Genes' = 'darkgoldenrod')

a <- ggplot(fig_a[class!='Proposed Novel Hotspot Genes' & class!='Hotspot Genes'],aes(x=tissue,y=mean,colour=class,ymin=min,ymax=max)) +
  geom_point(alpha=0.7,size=3,shape=15) +
  geom_linerange(alpha=0.5,size=3) + 
  geom_point(data=fig_a[class=='Hotspot Genes'],aes(x=tissue,y=mean,colour=class),size=5) +
  theme_pubr() +
  theme(axis.text = element_text(size=14), 
        axis.title = element_text(size=16), 
        axis.text.x = element_text(angle = 90, 
                                   vjust = 0.5, 
                                   hjust=1),
        legend.title = element_blank(), 
        legend.text = element_text(size=14)) +
  scale_colour_manual(values=a_fill) +
  ylim(0,1) +
  xlab('') +
  ylab('Proportion expressed genes (TPM > 1)')


### Figure 4BC: Distinct expression pattern in hotspot genes



In [ ]:
#Set seed
set.seed(1255)

brain_tissues <- c('Brain - Amygdala','Brain - Anterior cingulate cortex (BA24)','Brain - Caudate (basal ganglia)','Brain - Cerebellar Hemisphere','Brain - Cerebellum',
                   'Brain - Cortex','Brain - Frontal Cortex (BA9)','Brain - Hippocampus','Brain - Hypothalamus','Brain - Nucleus accumbens (basal ganglia)','Brain - Putamen (basal ganglia)',
                   'Brain - Spinal cord (cervical c-1)','Brain - Substantia nigra')

#Subset gene_tpm to randomly sampled genes, but additionally include all genes that contain ion channels.
#Group by tissue group (Brain, Other Tissues) and calculate the median TPM per tissue group.
median_gene_tpm <- gene_tpm %>%
  filter(class!='Control Genes' | hgnc %in% sampled_genes | contains_ion_channel==TRUE) %>%
  mutate(tissue_group = ifelse(tissue %in% brain_tissues,'Brain','Other Tissues')) %>%
  group_by(hgnc) %>%
  mutate(wilcox_p = wilcox.test(tpm ~ tissue_group,exact=FALSE)$p.value,
         wilcox_corrected_p = p.adjust(wilcox_p, method = 'bonferroni')) %>%
  group_by(hgnc,tissue_group) %>%
  mutate(median_tpm_tissuegroup = median(tpm)) %>%
  select(-c('tpm','tissue','ensembl_versioned')) %>%
  unique() %>%
  reshape2::dcast(formula=hgnc + class + contains_ion_channel + wilcox_p + wilcox_corrected_p ~ tissue_group, value.var='median_tpm_tissuegroup') %>%
  data.table()

#Calculate the proportion expressed higher in brain
fig_b <- median_gene_tpm %>%
  group_by(class) %>%
  summarise(
    num_brain = sum(Brain > `Other Tissues`, na.rm=T),
    num_other_tissues = sum(`Other Tissues` > Brain, na.rm=T),
    total = n(),
    num_unexpressed = total - (num_brain + num_other_tissues),
    prop_brain = sum(Brain > `Other Tissues`, na.rm=T) / n(),
    .groups='drop_last'
  ) %>%
  reshape2::melt(id.vars=c('class')) %>%
  data.table()

knitr::kable(fig_b)

#Fisher's exact: Hotspot genes versus control/DD consensus genes
hotspot_counts <- c(fig_b[class=='Hotspot Genes' & variable=='num_brain',value],
                    fig_b[class=='Hotspot Genes' & variable=='num_other_tissues',value])
dd_counts <- c(fig_b[class=='DD Consensus Genes' & variable=='num_brain',value],
                    fig_b[class=='DD Consensus Genes' & variable=='num_other_tissues',value])
control_counts <- c(fig_b[class=='Control Genes' & variable=='num_brain',value],
                    fig_b[class=='Control Genes' & variable=='num_other_tissues',value])

fisher.test(cbind(hotspot_counts,control_counts))
fisher.test(cbind(hotspot_counts,dd_counts))

#Calculate TPM differences between classes
fig_c <- median_gene_tpm %>%
  mutate(diff = Brain - `Other Tissues`) %>%
  mutate(class = ifelse(hgnc %in% ion_channel_genes & class=='DD Consensus Genes','PF00520-Containing DD Consensus Genes',
                        ifelse(hgnc %in% ion_channel_genes & class=='Control Genes','PF00520-Containing Control Genes',class)))

fig_c_supp <- gene_tpm %>%
  filter(class!='Control Genes' | hgnc %in% sampled_genes | contains_ion_channel==TRUE) %>%
  mutate(tissue_group = ifelse(tissue %in% brain_tissues,'Brain','Other Tissues')) %>%
  mutate(class = ifelse(hgnc %in% ion_channel_genes & class=='DD Consensus Genes','PF00520-Containing DD Consensus Genes',
                        ifelse(hgnc %in% ion_channel_genes & class=='Control Genes','PF00520-Containing Control Genes',class))) %>%
  filter(class %in% c('PF00520-Containing Control Genes','PF00520-Containing DD Consensus Genes','Hotspot Genes')) %>%
  mutate(class = factor(class, levels = c('PF00520-Containing Control Genes','PF00520-Containing DD Consensus Genes','Hotspot Genes')))

ggplot(fig_c_supp,aes(x=tissue,y=tpm,fill=class)) + 
  geom_boxplot(outlier.colour = NA, alpha = 0.8) +
  theme_pubr() +
  theme(axis.text = element_text(size=14), 
        axis.title = element_text(size=16), 
        axis.text.x = element_text(angle = 90, 
                                   vjust = 0.5, 
                                   hjust=1),
        legend.title = element_blank(), 
        legend.text = element_text(size=14)) +
  ylim(0,25) +
  xlab('') +
  ylab('TPM by Tissue') +
  scale_fill_manual(values=c('skyblue4','darkgreen','darkgoldenrod'))

ggplot(fig_c_supp[tpm > 1],aes(x=tissue,y=tpm,fill=class)) + 
  geom_boxplot(outlier.colour = NA, alpha = 0.8) +
  theme_pubr() +
  theme(axis.text = element_text(size=14), 
        axis.title = element_text(size=16), 
        axis.text.x = element_text(angle = 90, 
                                   vjust = 0.5, 
                                   hjust=1),
        legend.title = element_blank(), 
        legend.text = element_text(size=14)) +
  ylim(0,25) +
  xlab('') +
  ylab('TPM by Tissue (TPM > 1)') +
  scale_fill_manual(values=c('skyblue4','darkgreen','darkgoldenrod'))

#Figure 4B
b_x_labels = c('Control\nGenes','DD Consensus\nGenes','Hotspot\nGenes','Proposed Novel\nHotspot Genes')
b_levels = c('prop_unexpressed','prop_other','prop_brain')
b_fill = c('prop_unexpressed' = 'darkgrey',
         'prop_other' = 'steelblue3',
         'prop_brain' = 'steelblue4')
b_fill_labels = c('prop_unexpressed' = 'Unexpressed',
                'prop_other' = 'Other Tissues TPM > Brain TPM',
                'prop_brain' = 'Brain TPM > Other Tissues TPM')

b <- ggplot(fig_b[variable=='prop_brain'],aes(x=class,y=value,fill=factor(variable,levels=b_levels))) + 
  geom_col(colour='black',alpha=0.8) +
  theme_pubr() +
  theme(legend.position = 'none',
        axis.text = element_text(size=14), 
        axis.title = element_text(size=14), 
        legend.title = element_blank(), 
        legend.text = element_text(size=14)) +
  scale_fill_manual(labels=b_fill_labels,values=b_fill) +
  scale_x_discrete(labels=b_x_labels) +
  ylim(0,1) +
  xlab('') +
  ylab('Proportion of Genes')

#Figure 4C
c_levels = c('PF00520-Containing Control Genes','PF00520-Containing DD Consensus Genes','Hotspot Genes','Proposed Novel Hotspot Genes')
c_labels = c('PF00520-Containing Control Genes','PF00520-Containing DD Consensus Genes','Hotspot Genes')
c_x_labels = c('PF00520-Containing\nControl Genes','PF00520-Containing\nDD Consensus Genes','Hotspot Genes','Proposed Novel\nHotspot Genes')
c_fill = c('PF00520-Containing Control Genes' = 'skyblue3',
         'PF00520-Containing DD Consensus Genes' = 'darkgreen',
         'Hotspot Genes' = 'darkgoldenrod')

#Artificially sets the diff of TPCN1 for plotting to -10; true value -19.88
fig_c <- fig_c[,diff:=ifelse(class=='Proposed Novel Hotspot Genes' & diff < -10,-10,diff)]
  
c <- ggplot(fig_c[class!='Control Genes' & class!='DD Consensus Genes' & class!='Proposed Novel Hotspot Genes'],
              aes(x=factor(class,levels=c_levels),y=diff,colour=factor(class,levels=c_labels))) + 
  geom_boxplot(alpha=0.8, outlier.shape = NA) + 
  geom_point(data=fig_c[class=='Proposed Novel Hotspot Genes'],aes(x=3,y=diff),
             colour='black',
             shape=4,
             size=5,
             position = position_jitter(0.2)) +
  theme_pubr() +
  theme(legend.position = 'none',
        axis.text = element_text(size=14), 
        axis.title = element_text(size=14), 
        legend.title = element_blank(), 
        legend.text = element_text(size=14)) +
  scale_colour_manual(values=c_fill) +
  scale_x_discrete(labels=c_x_labels) +
  coord_cartesian(ylim=c(-10,20)) +
  ggsignif::geom_signif(
    comparisons = list(c('PF00520-Containing Control Genes','PF00520-Containing DD Consensus Genes'),c('PF00520-Containing DD Consensus Genes','Hotspot Genes')),
    map_signif_level = FALSE,
    y_position = c(12,17),
    colour='black') +
  xlab('') +
  ylab('TPM Brain - TPM Other Tissues')

#Figure 4
plots <- align_plots(a, b, align = 'v', axis = 'l')
bottom_row <- plot_grid(plots[[2]], c, labels = c('B', 'C'), label_size = 14, rel_widths = c(1,1.55))
plot_grid(plots[[1]], bottom_row, labels = c('A', ''), label_size = 14, ncol = 1, rel_heights = c(1.75,1))
  


**Figure 4. Hotspot genes have a distinct gene expression pattern.** *A) Hotspot genes share a characteristic gene expression pattern across tissues.* Expression of the 19 established DD genes containing a mutation hotspot (hotspot genes, yellow) was evaluated across 54 GTEx tissues. Hotspot gene expression was compared to DD genes (DD consensus, discordant, and novel genes, blue; n = 991) and a randomly sampled set of 1000 GENCODE v37 genes not associated with intellectual disability or developmental delay (control genes, blue). The proportion of expressed genes was calculated based on the median transcripts per million (TPM) per gene in each of the 54 tissues; TPM > 1 was considered expressed. For appropriate comparison to hotspot genes, DD and control genes were randomly sampled into sets of 25 (1000 iterations) and the proportion of expressed genes was computed per set. The mean (square) and standard deviation (bars) of the distribution across these sets is shown in Figure 4A. *B) Hotspot genes are highly expressed in brain.* Given the distinct profile of hotspot gene expression between brain tissues and most other GTEx tissues, we subset the data into two tissue groups (brain and other tissues; see Methods). The median TPM per gene was calculated for each tissue group, and these median TPMs were compared to determine the proportion of the control, DD, hotspot, and proposed novel hotspot gene sets with higher expression in brain than whole blood (dark blue). The remaining genes in each set either have higher expression in other tissues than brain (light blue; Supplementary Figure N1) or are constitutively unexpressed (grey, Supplementary Figure N1). Hotspot genes have a significantly higher proportion of genes with expression in brain > other tissues when compared with control genes (p = 2.985 x 10-5, Fisher’s exact test) and DD consensus genes (p = 0.002). Of the proposed novel hotspot genes, 3 (*CACNA1B*, *KCNH5*, and *KCNG1*) share this property. *C) Hotspot genes are overexpressed in brain compared to other DD ion channel-containing genes.* Because all identified hotspot genes contain a PF00520 ion channel domain, we tested whether the difference between brain TPM and other tissue TPM (differential expression) could differentiate hotspot genes from other PF00520 domain-containing genes. Positive values on the y-axis indicate higher expression in brain, and negative values indicate higher expression in other tissues. PF00520-containing DD genes have significantly higher differential expression compared to PF00520-containing control genes (p = 0.019, Wilcoxon rank-sum test). Hotspot genes have significantly higher differential expression when compared to other PF00520-containing DD genes (p = 0.039, Wilcoxon rank-sum test). Black crosses on the hotspot distribution indicate differential expression of named proposed novel hotspot genes (n = 6) between tissue groups.

### Figure 5: Constraint


In [ ]:
#Get relevant constraint information
probabilistic_constraint <- c('pLI','pRec','pNull')
upper_bound_fraction <- c('oe_lof_upper','oe_mis_upper','oe_syn_upper')
observed_expected <- c('oe_lof','oe_mis','oe_syn','oe_lof_lower','oe_mis_lower','oe_syn_lower',upper_bound_fraction)
z_scores <- c('lof_z','mis_z','syn_z')
all_constraint <- unique(c(probabilistic_constraint,upper_bound_fraction,observed_expected,z_scores))

constraint <- constraint_in %>%
  subset(select=c('gene',all_constraint)) %>%
  reshape2::melt(id.vars = 'gene', value.name = 'value', var.name = 'constraint_metric') %>%
  filter(!is.na(value)) %>%
  mutate(class=ifelse(gene %in% novel_hotspot_genes,'Proposed Novel Hotspot Genes',
                      ifelse(gene %in% hotspot_genes,'Hotspot Genes',
                             ifelse(gene %in% dd_genes,'DD Consensus Genes','Control Genes'))))

#Constraint counts
constraint_table <- constraint %>%
  select(c('gene','class')) %>%
  unique() %>%
  group_by(class) %>%
  count()

colnames(constraint_table) <- c('Class','Count')

knitr::kable(constraint_table)

constraint_table_2 <- constraint_in %>%
  subset(select=c('gene',all_constraint)) %>%
  filter(gene %in% hotspot_genes | gene %in% novel_hotspot_genes)

#Figure 5A
fig_5a <- constraint %>%
  filter(variable %in% observed_expected) %>%
  group_by(class,variable) %>%
  mutate(variable=gsub('oe_','',variable)) %>%
  mutate(type=substr(variable,1,3)) %>%
  mutate(bound=ifelse(grepl('upper',variable),'upper',
                      ifelse(grepl('lower',variable),'lower',''))) %>%
  group_by(class,type,bound) %>%
  summarise(
    mean = mean(value),
    .groups = 'drop_last'
  ) %>%
  mutate(bound=ifelse(bound=='','oe',bound)) %>%
  reshape2::dcast(formula=class + type ~ bound, value.var='mean')

a_labels = c('Loss of function','Missense','Synonymous')
a_levels = c('Control Genes','DD Consensus Genes','Hotspot Genes','Proposed Novel Hotspot Genes')
a_x_labels = c('Control Genes','DD Consensus Genes','Hotspot Genes','Proposed Novel\nHotspot Genes')

a <- ggplot(fig_5a,aes(x=factor(class,levels=a_levels),y=oe,colour=type,ymin=lower,ymax=upper)) + 
  geom_pointrange(position=position_dodge(width=1),size=1.6) + 
  geom_hline(yintercept=1,linetype='dashed',colour='black') +
  theme_pubr() +
  theme(axis.text = element_text(size=14), 
        axis.title = element_text(size=16), 
        legend.title = element_blank(), 
        legend.text = element_text(size=14)) +
  scale_x_discrete(labels=a_x_labels) +
  scale_colour_manual(labels=a_labels, values=c('tomato4','darkorange3','deeppink4')) + 
  ylab('Observed/expected counts') +
  xlab('')


In [ ]:
#Get global missense constraint
global_oe_mis <- constraint %>%
  filter(variable=='oe_mis')

#Add global missense constraint to regional constraint infile and parse
fig_5b <- regional_constraint %>%
  inner_join(global_oe_mis,by='gene') %>%
  mutate(family=ifelse(grepl('CACNA1',gene),'CACNA1\nGene Family',
                             ifelse(grepl('KCN',gene),'KCN\nGene Family',
                                    ifelse(grepl('SCN',gene),'SCN\nGene Family','Other')))) %>%
  dplyr::select(c('obs_exp','value','gene','family')) %>%
  unique() %>%
  data.table()

#Figure 5B
b_levels = c('CACNA1\nGene Family','KCN\nGene Family','SCN\nGene Family','Other')
b_colour = c('CACNA1\nGene Family' = 'darkolivegreen',
             'KCN\nGene Family' = 'bisque2',
             'SCN\nGene Family' = 'lightpink4',
             'Other' = 'chocolate4')

b <- ggplot(fig_5b,aes(x=value,y=obs_exp,colour=factor(family,levels=b_levels))) + 
  geom_point(size=7) +
  geom_abline(slope=1,intercept=0,colour='black',linetype='dashed') +
  theme_pubr() +
  theme(axis.text = element_text(size=14), 
        axis.title = element_text(size=16),
        legend.position = 'top',
        legend.text = element_text(size=14),
        legend.title = element_blank()) +
  scale_colour_manual(values=b_colour) +
  xlim(0,1) +
  ylim(0,1) + 
  xlab('Global missense constraint') +
  ylab('Regional missense constraint at hotspots')

#Figure 5
plot_grid(a,b,labels='AUTO',nrow=2,ncol=1,rel_widths = c(2,1),label_size = 14)

#Put a p-value on regional missense constraint by comparing hotspot domains to ion channel domains with no statistical evidence of being a mutation hotspot
mpc <- mpc_in[,mpc_starts:=ifelse(genomic_start<genomic_end,genomic_start,genomic_end)]
mpc <- mpc[,mpc_ends:=ifelse(genomic_start<genomic_end,genomic_end,genomic_start)]
mpc <- mpc[,obs_exp:=as.numeric(sub(',','.',obs_exp))]
mpc <- mpc[,gene_name:=gene]
mpc <- mpc[,gencode_transcript:=transcript]
mpc <- mpc[,c('gene_name','gencode_transcript','mpc_starts','mpc_ends','obs_exp','region_name')]

all_ion_channel_domains <- fread('/Users/jh/Downloads/ion_channels.csv')
all_ion_channel_domains <- all_ion_channel_domains[,c('gene_name','chromosome','chromosome_position','codon_base_pair_position','gencode_transcription_id','strand')]

getOverlaps <- function( gene, transcript ) {
  
  mpc_temp <- mpc[gene_name==gene & gencode_transcript==transcript]
  
  temp_ion_channel_domains <- all_ion_channel_domains[gene_name==gene & gencode_transcription_id==transcript]
  strand =  unique(temp_ion_channel_domains[,strand])

  if( strand=='minus' ) {
    
      temp_ion_channel_domains_sorted <- temp_ion_channel_domains[order(chromosome_position,decreasing = TRUE)]
    
  } else {
    
      temp_ion_channel_domains_sorted <- temp_ion_channel_domains[order(chromosome_position,decreasing = FALSE)]
    
  }
  
  assignment_vec <- rep(c(rep('start',3),rep('stop',3)),nrow(temp_ion_channel_domains)/6)
  temp_ion_channel_domains_sorted <- temp_ion_channel_domains_sorted[,assignment:=assignment_vec]
  
  if( strand=='minus' ) {
    
    end <- temp_ion_channel_domains_sorted[gene_name==gene & gencode_transcription_id==transcript & assignment=='start' & codon_base_pair_position==0,chromosome_position]
    start <- temp_ion_channel_domains_sorted[gene_name==gene & gencode_transcription_id==transcript & assignment=='stop'& codon_base_pair_position==2,chromosome_position]
    
  } else {
    
    start <- temp_ion_channel_domains_sorted[gene_name==gene & gencode_transcription_id==transcript & assignment=='start' & codon_base_pair_position==0,chromosome_position]
    end <- temp_ion_channel_domains_sorted[gene_name==gene & gencode_transcription_id==transcript & assignment=='stop'& codon_base_pair_position==2,chromosome_position]
    
  }
  
  domains_temp <- data.table(gene = gene, transcript = transcript, domain_starts = start, domain_ends = end)
  
  setkey(mpc_temp,mpc_starts,mpc_ends)
  setkey(domains_temp,domain_starts,domain_ends)
  
  overlaps <- foverlaps(domains_temp, mpc_temp, nomatch = 0)
  
  overlaps <- overlaps[,c('gene_name','gencode_transcript','obs_exp','region_name','mpc_starts','mpc_ends','domain_starts','domain_ends')]
  return(overlaps)
  
}

genes <- unique(all_ion_channel_domains[,gene_name])
overlaps_out <- vector('list',length=nrow(all_ion_channel_domains))
n = 1

for( gene in genes ) {
  
  transcripts <- unique(all_ion_channel_domains[gene_name==gene,gencode_transcription_id])
  
  for( transcript in transcripts ) {
    
    overlaps_out[[n]] <- getOverlaps(gene, transcript)
    n = n + 1
    
  }
  
}

overlaps <- do.call(rbind,overlaps_out)

overlaps <- overlaps[,class:=ifelse(gene_name %in% hotspot_genes,'Hotspot Genes',
                                    ifelse(!gene_name %in% c(hotspot_genes, novel_hotspot_genes, dd_genes),'Control Genes','Other'))]

overlaps_to_test <- overlaps[class=='Hotspot Genes' | class=='Control Genes']

overlaps_labels <- c('PF00520 Domain-Containing\nControl Genes','Hotspot Genes')

ggplot(overlaps_to_test,aes(x=class,y=obs_exp,fill=class)) + 
  geom_boxplot(alpha=0.8) +
  theme_pubr() +
  theme(legend.position = 'none',
        axis.text = element_text(size=14), 
        axis.title = element_text(size=14), 
        legend.title = element_blank(), 
        legend.text = element_text(size=14)) +
  ggsignif::geom_signif(
    comparisons = list(c('Hotspot Genes','Control Genes')),
    y_position = 1.3,
    colour='black',
    test = 'wilcox.test') +
  scale_x_discrete(labels=overlaps_labels) +
  scale_fill_manual(values=c('skyblue3','darkgoldenrod')) +
  ylab('Observed/expected regional missense constraint at PF00520 domains') +
  xlab('')
  


**Figure 5. Proposed novel hotspot genes are constrained against loss of function and missense variation.** *A) Hotspot genes and proposed novel hotspot genes show population evidence of constraint.* The observed variant counts for loss of function (red), missense (orange), and synonymous (pink) variants from the gnomAD v2 release were compared to the expected counts based on a null mutational model (Samocha et. al., 2017). Points represent the mean observed/expected ratio for all genes in each set and bars denote the mean upper and lower bound fractions for these ratios. The dashed line at observed/expected = 1 indicates perfect adherence to the null mutational model (observed counts = expected counts); values that fall below this line are constrained. *B) Mutation hotspots occur in missense constrained regions within genes.* While the missense constraint values in Figure 5A are calculated across the entire coding region, some genes (n = 2700) show evidence of regional missense constraint (Samocha et. al., 2019). These genes can be statistically divided into regions with different observed/expected ratios for missense variation. 19 out of 25 hotspot and proposed novel hotspot genes show evidence of regional missense constraint (16 hotspot genes and 3 proposed novel hotspot genes). Points represent regions of differential missense constraint that contain mutation hotspots in these genes coloured by gene family. The global missense constraint score (observed/expected counts across the coding region, x-axis) is compared against the regional missense constraint score of these regions (observed/expected counts across differentially constrained region, y-axis). The dashed line at regional observed/expected = genic observed/expected indicates a regional missense constraint equal to the overall constraint score of the gene; points that fall below it have higher constraint against missense variation at mutation hotspots. 

### Supplementary figures


In [ ]:
#Supplement A

fig_a_supp <- gene_tpm %>%
  filter(class!='Control Genes' | hgnc %in% sampled_genes | contains_ion_channel==TRUE) %>%
  filter(contains_ion_channel==TRUE) %>%
  filter(class!='Proposed Novel Hotspot Genes') %>%
  group_by(tissue,class) %>%
  summarise(
    prop_expressed = sum(tpm > 1, na.rm=T) / n(),
    .groups = 'keep'
  ) %>%
  data.table()

#Figure 4A
a_supp_colour <- c('Control Genes' = 'skyblue3',
            'DD Consensus Genes' = 'darkgreen',
            'Hotspot Genes' = 'darkgoldenrod')

a_supp_labels <- c('Control Genes' = 'PF00520 Domain-Containing Control Genes',
            'DD Consensus Genes' = 'PF00520 Domain-Containing NDD-Associated Genes',
            'Hotspot Genes' = 'Hotspot Genes')

a <- ggplot(fig_a_supp,aes(x=tissue,y=prop_expressed,colour=class)) +
  geom_point(size=4) +
  theme_pubr() +
  theme(axis.text = element_text(size=14), 
        axis.title = element_text(size=16), 
        axis.text.x = element_text(angle = 90, 
                                   vjust = 0.5, 
                                   hjust=1),
        legend.title = element_blank(), 
        legend.text = element_text(size=14)) +
  scale_colour_manual(values=a_supp_colour,labels=a_supp_labels) +
  ylim(0,1) +
  xlab('') +
  ylab('Proportion expressed genes (TPM > 1)')

#Supplement N
fig_b_supp_1 <- median_gene_tpm %>%
  group_by(class) %>%
  summarise(
    prop_brain = sum(Brain > `Other Tissues`, na.rm=T) / n(),
    prop_other = sum(`Other Tissues` > Brain, na.rm=T) / n(),
    prop_unexpressed = sum(Brain == `Other Tissues`, na.rm=T) / n(),
    .groups='drop_last'
  ) %>%
  reshape2::melt(id.vars=c('class')) %>%
  data.table()

#Supplement N
fig_b_supp_2 <- median_gene_tpm %>%
  filter(contains_ion_channel==TRUE) %>%
  group_by(class) %>%
  summarise(
    num_brain = sum(Brain > `Other Tissues`, na.rm=T),
    num_other_tissues = sum(`Other Tissues` > Brain, na.rm=T),
    total = n(),
    num_unexpressed = total - (num_brain + num_other_tissues),
    prop_brain = sum(Brain > `Other Tissues`, na.rm=T) / n(),
    prop_other = sum(`Other Tissues` > Brain, na.rm=T) / n(),
    prop_unexpressed = sum(Brain == `Other Tissues`, na.rm=T) / n(),
    .groups='drop_last'
  ) %>%
  reshape2::melt(id.vars=c('class')) %>%
  data.table()

#Fisher's exact: Hotspot genes versus ion channel containing control/DD consensus genes
hotspot_counts_PF00520 <- c(fig_b_supp_2[class=='Hotspot Genes' & variable=='num_brain',value],
                    fig_b_supp_2[class=='Hotspot Genes' & variable=='num_other_tissues',value])
dd_counts_PF00520 <- c(fig_b_supp_2[class=='DD Consensus Genes' & variable=='num_brain',value],
                       fig_b_supp_2[class=='DD Consensus Genes' & variable=='num_other_tissues',value])
control_counts_PF00520 <- c(fig_b_supp_2[class=='Control Genes' & variable=='num_brain',value],
                    fig_b_supp_2[class=='Control Genes' & variable=='num_other_tissues',value])

fisher.test(cbind(hotspot_counts_PF00520,control_counts_PF00520))
fisher.test(cbind(hotspot_counts_PF00520,dd_counts_PF00520))

#Supplement N
bsupp_x_labels = c('PF00520-Containing\nControl Genes','PF00520-Containing\nDD Consensus Genes','Hotspot Genes','Proposed Novel\nHotspot Genes')
bsupp_fill = c('prop_unexpressed' = 'darkgrey',
         'prop_other' = 'steelblue3',
         'prop_brain' = 'steelblue4')
bsupp_levels = c('prop_unexpressed','prop_other','prop_brain')
bsupp_fill_labels = c('prop_unexpressed' = 'Unexpressed',
                'prop_other' = 'Other Tissues TPM > Brain TPM',
                'prop_brain' = 'Brain TPM > Other Tissues TPM')

ggplot(fig_b_supp_2[variable %in% c('prop_brain','prop_other','prop_unexpressed')],aes(x=class,y=value,fill=factor(variable,levels=bsupp_levels))) + 
  geom_col(colour='black',alpha=0.8) +
  theme_pubr() +
  theme(legend.position = 'top',
        axis.text = element_text(size=14), 
        axis.title = element_text(size=14), 
        legend.title = element_blank(), 
        legend.text = element_text(size=14)) +
  scale_fill_manual(labels=bsupp_fill_labels,values=bsupp_fill) +
  scale_x_discrete(labels=bsupp_x_labels) +
  xlab('') +
  ylab('Proportion of Genes')

#Supplmenetary table N
median_gene_tpm_table <- median_gene_tpm %>%
  filter(class=='Hotspot Genes' | class=='Proposed Novel Hotspot Genes') %>%
  select(-c('contains_ion_channel')) %>%
  plyr::arrange(class,wilcox_corrected_p) %>%
  data.table()

colnames(median_gene_tpm_table) <- c('Gene','Class','Wilcox P','Bonferroni-Corrected Wilcox P','Median TPM Brain','Median TPM Other Tissues')

knitr::kable(median_gene_tpm_table)

#Supplement N
total_genes_rmc = length(unique(mpc_in[,gene]))
dd_consensus_with_rmc = length(unique(mpc_in[gene %in% dd_genes,gene]))
hotspot_with_rmc = length(unique(mpc_in[gene %in% hotspot_genes,gene]))
novel_hotspot_with_rmc = length(unique(mpc_in[gene %in% novel_hotspot_genes,gene]))
control_genes_with_rmc = total_genes_rmc - (dd_consensus_with_rmc + hotspot_with_rmc + novel_hotspot_with_rmc)

hotspot_counts <- c(hotspot_with_rmc,length(hotspot_genes))
dd_consensus_counts <- c(dd_consensus_with_rmc,length(dd_genes))
control_counts = c(control_genes_with_rmc,length(unique(gene_tpm[class=='Control Genes',hgnc])))

supp_c <- data.table(rmc_counts=c(novel_hotspot_with_rmc,hotspot_with_rmc,dd_consensus_with_rmc,control_genes_with_rmc),
                     total_counts=c(length(novel_hotspot_genes),length(hotspot_genes),length(dd_genes),length(unique(gene_tpm[class=='Control Genes',hgnc]))),
                     class=c('Proposed Novel Hotspot Genes','Hotspot Genes','DD Consensus Genes','Control Genes'))

supp_c <- supp_c %>%
  mutate(prop_rmc = rmc_counts/total_counts,
         prop_other = 1 - prop_rmc) %>%
  reshape2::melt(id.vars='class') %>%
  data.table()

supp_c_x_labels = c('Control Genes','DD Consensus Genes','Hotspot Genes','Proposed Novel\nHotspot Genes')
supp_c_labels = c('Other Genes','Proportion of Genes with Evidence of Regional Missense Constraint')
supp_c_fill = c('darkgrey','tomato4')

ggplot(supp_c[variable=='prop_rmc' | variable=='prop_other'],
       aes(x=class,y=value,fill=factor(variable,levels=c('prop_other','prop_rmc')))) +
  geom_col() +
  theme_pubr() +
  theme(legend.position = 'top',
        axis.text = element_text(size=14), 
        axis.title = element_text(size=14), 
        legend.title = element_blank(), 
        legend.text = element_text(size=14)) +
  scale_fill_manual(values=supp_c_fill,labels=supp_c_labels) +
  scale_x_discrete(labels=supp_c_x_labels) +
  xlab('') +
  ylab('Proportion of Genes')

fisher.test(cbind(hotspot_counts,control_counts))
fisher.test(cbind(hotspot_counts,dd_consensus_counts))
fisher.test(cbind(dd_consensus_counts,control_counts))

#Enrichment of activating mutations analysis on DDG2P

gene_list <- fread('/Users/jh/Resources/Gene_Lists/DDG2P_22-4-2021.csv')
nonhap_list <- c('dominant negative','gain of function','activating','increased gene dosage')

ddg2p_genes <- length(unique(gene_list[,`gene symbol`]))
ddg2p_activating_genes <- length(unique(gene_list[`mutation consequence` %in% nonhap_list,`gene symbol`]))
hotspot_ddg2p_genes <- length(unique(gene_list[`gene symbol` %in% hotspot_genes,`gene symbol`]))
hotspot_ddg2p_activating_genes <- length(unique(gene_list[`gene symbol` %in% hotspot_genes & `mutation consequence` %in% nonhap_list,`gene symbol`]))

fisher.test(cbind(c(ddg2p_activating_genes,(ddg2p_genes - ddg2p_activating_genes)),
                  c(hotspot_ddg2p_activating_genes,(hotspot_ddg2p_genes - hotspot_ddg2p_activating_genes))))

#NDD genes with ion channel domain in the lenient set -- for discussion

test <- median_gene_tpm %>%
  filter(class!='Control Genes' & class!='Proposed Novel Hotspot Genes' & contains_ion_channel==TRUE) %>%
  mutate(class = ifelse(class=='DD Consensus Genes' & contains_ion_channel==TRUE,'NDD-Associated PF00520 Domain-Containing Genes',class)) %>%
  reshape2::melt(measure.vars=c('Brain','Other Tissues')) %>%
  data.table()

ggplot(test,aes(x=class,y=value,fill=variable)) + 
  geom_boxplot() +
  theme_pubr()

#Test of highest TPM tissue

max_tpm_tissue <- gene_tpm %>%
  group_by(hgnc) %>%
  filter(tpm==max(tpm)) %>%
  mutate(tissue_type = ifelse(tissue %in% brain_tissues,'Brain','Other Tissues')) %>%
  ungroup() %>%
  group_by(class) %>%
  summarise(
    prop_brain = sum(tissue_type=='Brain', na.rm=T) / n(),
    prop_other = sum(tissue_type=='Other Tissues', na.rm=T) / n(),
    .groups = 'keep'
  ) %>%
  data.table()

brain_expressed_genes <- gene_tpm %>%
  group_by(ensembl_versioned) %>%
  filter(tissue %in% brain_tissues & tpm > 1) %>%
  ungroup() %>%
  pull(hgnc)
  
num_other_expressed_tissues <- gene_tpm %>%
  mutate(class = ifelse(contains_ion_channel==TRUE,paste0(class,' - TRUE'),class)) %>%
  filter(tpm > 1) %>%
  group_by(ensembl_versioned) %>%
  mutate(num_expressed = n(),
         prop_expressed = num_expressed/41) %>%
  data.table()

num_other_expressed_tissues_brain <- gene_tpm %>%
  filter(hgnc %in% brain_expressed_genes) %>%
  filter(!tissue %in% brain_tissues) %>%
  mutate(class = ifelse(contains_ion_channel==TRUE,paste0(class,' - TRUE'),class)) %>%
  filter(tpm > 1) %>%
  group_by(ensembl_versioned) %>%
  mutate(num_expressed = n(),
         prop_expressed = num_expressed/41) %>%
  data.table()
  
labels_1 = c('Control Genes' = 'Control Genes\nExpressed',
           'Control Genes - TRUE' = 'PF00520 Domain-Containing\nControl Genes Expressed',
           'DD Consensus Genes' = 'NDD-Associated Genes\nExpressed',
           'DD Consensus Genes - TRUE' = 'PF00520 Domain-Containing\nNDD-Associated Genes\nExpressed',
           'Hotspot Genes - TRUE' = 'Hotspot Genes\nExpressed',
           'Proposed Novel Hotspot Genes - TRUE' = 'Proposed Novel\nHotspot Genes Expressed')

labels_2 = c('Control Genes' = 'Control Genes\nExpressed in Brain',
           'Control Genes - TRUE' = 'PF00520 Domain-Containing\nControl Genes Expressed in Brain',
           'DD Consensus Genes' = 'NDD-Associated Genes\nExpressed in Brain',
           'DD Consensus Genes - TRUE' = 'PF00520 Domain-Containing\nNDD-Associated Genes\nExpressed in Brain',
           'Hotspot Genes - TRUE' = 'Hotspot Genes\nExpressed in Brain',
           'Proposed Novel Hotspot Genes - TRUE' = 'Proposed Novel\nHotspot Genes Expressed in Brain')

supp_a <- ggplot(num_other_expressed_tissues,aes(x=class,y=num_expressed,fill=class)) +
  geom_boxplot(alpha=0.8) + 
  theme_pubr() +
  theme(legend.position = 'none',
        axis.text = element_text(size=14), 
        axis.title = element_text(size=14), 
        legend.title = element_blank(),
        legend.text = element_text(size=14)) +
  scale_x_discrete(labels=labels_1) +
  scale_fill_manual(values=c('skyblue3','skyblue3','darkgreen','darkgreen','darkgoldenrod','darkgoldenrod'),labels=labels_1) +
  ggsignif::geom_signif(
    comparisons = list(c('Control Genes','Control Genes - TRUE'),c('DD Consensus Genes','DD Consensus Genes - TRUE'),
                       c('DD Consensus Genes - TRUE','Hotspot Genes - TRUE')),
    map_signif_level = FALSE,
    y_position = c(45,45,43),
    colour='black') +
  xlab('') +
  ylab('Number of Tissues with Expression (TPM > 1)')

supp_b <- ggplot(num_other_expressed_tissues_brain,aes(x=class,y=num_expressed,fill=class)) +
  geom_boxplot(alpha=0.8) + 
  theme_pubr() +
  theme(legend.position = 'none',
        axis.text = element_text(size=14), 
        axis.title = element_text(size=14), 
        legend.title = element_blank(),
        legend.text = element_text(size=14)) +
  scale_x_discrete(labels=labels_2) +
  scale_fill_manual(values=c('skyblue3','skyblue3','darkgreen','darkgreen','darkgoldenrod','darkgoldenrod'),labels=labels_2) +
  ggsignif::geom_signif(
    comparisons = list(c('Control Genes','Control Genes - TRUE'),c('DD Consensus Genes','DD Consensus Genes - TRUE'),
                       c('DD Consensus Genes - TRUE','Hotspot Genes - TRUE')),
    map_signif_level = FALSE,
    y_position = c(45,45,43),
    colour='black') +
  xlab('') +
  ylab('Number of Non-Brain Tissues with Expression (TPM > 1)')

plot_grid(supp_a,supp_b,nrow=2,labels='AUTO')


skeletal_muscle_genes <- c('SCN4A','RYR1','CACNA1S','KCNA7')

test <- gene_tpm %>%
  filter(hgnc %in% skeletal_muscle_genes) %>%
  filter(tpm > 1) %>%
  data.table()

ggplot(test,aes(x=tissue,y=tpm,colour=hgnc)) + 
  geom_point(size=5,alpha=0.6) +
  geom_hline(yintercept=1,linetype='dashed',colour='black') +
  theme_pubr() +
  theme(axis.text = element_text(size=14), 
        axis.title = element_text(size=16), 
        axis.text.x = element_text(angle = 90, 
                                   vjust = 0.5, 
                                   hjust=1),
        legend.title = element_blank(), 
        legend.text = element_text(size=14)) +
  xlab('') +
  ylab('TPM of Expressed Genes (TPM > 1)')
